In [43]:
from StatsAggregation.common import paragraphs_limmited_by_symbols
import sys
import unicodedata
import os
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import pymorphy2

In [2]:
writers_dir = input()

C:\Users\annag\Documents\MLDS_writers


In [14]:
data = {}
for writer in os.listdir(writers_dir):
    paras = paragraphs_limmited_by_symbols(writer, writers_dir, symbol_lim=3000)
    texts = ['\n'.join(para) for para in paras]
    data[writer] = texts

In [15]:
for k, v in data.items():
    print(f'{k}: {len(v)}')

Аверченко: 1036
Булгаков: 976
Бунин: 1000
Гоголь: 476
Горький: 1754
Достоевский: 3025
Карамзин: 10
Куприн: 993
Лермонтов: 153
Мордовцев: 556
Пушкин: 473
Салтыков-Щедрин: 917
Толстой: 1375
Тургенев: 243
Чехов: 262


In [18]:
pre_df = []
for k in data:
    for v in data[k]:
        pre_df.append([k, v])
df = pd.DataFrame(pre_df, columns=['author', 'text'])

In [19]:
df.head()

,author,text
0,Аверченко,Мы за пять лет. Материалы [к биографии]\nКак б...
1,Аверченко,Сидел он за большим письменным столом перед де...
2,Аверченко,"— Да уж, — качал головой сдержанный Ре-ми. — Н..."
3,Аверченко,"Поверит ли кто-нибудь, что нами за эти пять ле..."
4,Аверченко,"8\nАверченко А. Избранные рассказы. М., 1985. ..."


In [20]:
df.to_csv('author_text.csv', index=False)

In [28]:
punkt = ''.join([chr(i) for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')])

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\annag\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [30]:
def remove_punkt(text):
    tokens = word_tokenize(text, language='russian')
    tokens = [token.strip(punkt) for token in tokens]
    return ' '.join([token for token in tokens if token])

In [55]:
df_no_punkt = df.copy()

df_no_punkt['text_no_punkt'] = df_no_punkt['text'].apply(remove_punkt)
df_no_punkt.drop('text', axis=1, inplace=True)

In [66]:
df_no_punkt.to_csv('author_text_no_punkt.csv', index=False)

In [47]:
def lemmatize_with_tags(text):
    sw = set(stopwords.words('russian'))
    morph = pymorphy2.MorphAnalyzer()
    lemmas = []
    tags = []
    for w in text.split():
        if w not in sw:
            anls = morph.parse(w)[0]
            lemmas.append(anls.normal_form)
            tags.append(f'{len(w)}_{anls.tag.POS}')
    return ' '.join(lemmas), ' '.join(tags)

In [48]:
lemmas_and_tags = list(map(lemmatize_with_tags, df_no_punkt['text']))

In [51]:
df_lemmas_and_tags = pd.DataFrame(lemmas_and_tags, columns=['lemmas', 'tags'])
df_lemmas_and_tags['author'] = df['author']
df_lemmas_and_tags = df_lemmas_and_tags[['author', 'lemmas', 'tags']]

In [52]:
df_lemmas_and_tags

,author,lemmas,tags
0,Аверченко,мы пять год материал биография как кроваво-кра...,2_NPRO 4_NUMR 3_NOUN 9_NOUN 9_NOUN 3_CONJ 15_A...
1,Аверченко,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...
2,Аверченко,да качать голова сдержать ре-ми нехорошо нехор...,2_PRCL 5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_A...
3,Аверченко,поверить кто-нибудь мы пять год совместно м.г ...,7_VERB 10_NPRO 4_NPRO 4_NUMR 3_NOUN 9_ADVB 3_N...
4,Аверченко,8 аверченко а избранный рассказ м 1985 с 7 9 е...,1_None 9_NOUN 1_CONJ 9_ADJF 8_NOUN 1_NOUN 4_No...
...,...,...,...
13244,Чехов,vii быть восьмой час вечер из передний крыльцо...,3_None 3_VERB 7_ADJF 3_NOUN 6_NOUN 2_PREP 8_AD...
13245,Чехов,мы помолчать я мысленно сделать расчёт сказать...,2_NPRO 9_VERB 1_NPRO 8_ADVB 6_VERB 6_NOUN 6_VE...
13246,Чехов,1891 чехов а.п жена чехов а.п.—м да медиа 2015...,4_None 5_NOUN 3_None 4_NOUN 5_NOUN 6_NOUN 2_PR...
13247,Чехов,хрронический катар желудок зубрило студент шаг...,12_ADJF 5_NOUN 7_NOUN 6_NOUN 7_NOUN 5_GRND 4_N...


In [54]:
df_lemmas_and_tags.to_csv('author_lemmas_tags.csv', index=False)

In [58]:
df_tokens = df.copy()

df_tokens['tokens'] = [' '.join(word_tokenize(text, language='russian')) for text in df['text']]
df_tokens.drop('text', axis=1, inplace=True)

In [60]:
df_tokens.to_csv('author_tokens.csv', index=False)

In [62]:
aggregated_df = pd.concat([df[['author']]] + [x.iloc[:, 1:] for x in (df, df_no_punkt, df_lemmas_and_tags, df_tokens)], axis=1)

In [64]:
aggregated_df.to_csv('aggregated_df.csv', index=False)